In [2]:
import functions as volt
import pandas as pd
import numpy as np
import time

def getEarningsImpact():
    QQQCompanies = volt.getCompaniesETF('QQQ')['asset'].tolist()
    spyCompanies = volt.getCompaniesETF('SPY')['asset'].tolist()
    spyQQQSymbols = set(QQQCompanies + spyCompanies)

    earningsCalendar = volt.getEarningsCalendar()
    potentialUSSymbols = earningsCalendar[earningsCalendar['symbol'].str.contains(r'^[A-Za-z]+$', na=False)]

    USSymbols = potentialUSSymbols[potentialUSSymbols['symbol'].isin(spyQQQSymbols)]

    impliedVolatilites = volt.get30dayIVList(USSymbols['symbol'].tolist())
    time.sleep(60)

    df = pd.DataFrame(list(impliedVolatilites.items()),columns=['ticker','IV'])
    df = df[df['IV'] != 1.0].sort_values('IV',ascending=False)
    df = df.dropna()

    if len(df) < 5:
        df = df[0:len(df)]
    else:
        df = df[0:5]

    df = df.reset_index(drop=True)
    for i in df.index:
        ticker = df.iloc[i]['ticker']
        date = earningsCalendar[earningsCalendar['symbol'] == ticker]['date'].values[0]
        df.loc[i, 'date'] = date
        df.loc[i, 'impliedMove'] = volt.getImpliedMove(ticker)

    return df.dropna()


In [2]:
earningsImpact = getEarningsImpact()

BK
0.22688304997213266
WFC
nan
FAST
nan
BLK
nan
JPM
nan
54.823938
PGR
0.3008500686306374
STT
0.3066821829788136
WBA
0.873725070101885
JNJ
0.23267588598444555
JBHT
0.3577819143199087
27.677688000000003
  ticker        IV
0     BK  0.226883
1    WFC       NaN
2   FAST       NaN
3    BLK       NaN
4    JPM       NaN
5    PGR  0.300850
6    STT  0.306682
7    WBA  0.873725
8    JNJ  0.232676
9   JBHT  0.357782


In [3]:
earningsImpact

,ticker,IV,date,impliedMove
0,WBA,0.873725,2024-10-15,0.146580
1,JBHT,0.357782,2024-10-15,0.052015
2,STT,0.306682,2024-10-15,0.051764
3,PGR,0.300850,2024-10-11,0.047621
4,JNJ,0.232676,2024-10-15,0.027004


In [33]:
x = {'WBA':0.874193,'DFS':0.744820,'UAL':0.714157,'POOL':0.709247,'TFC':0.658761}

earningsCalendar = volt.getEarningsCalendar()

df = pd.DataFrame(list(x.items()),columns=['ticker','IV'])

for i in df.index:
    ticker = df.iloc[i]['ticker']
    date = earningsCalendar[earningsCalendar['symbol'] == ticker]['date'].values[0]
    df.loc[i, 'date'] = date

,ticker,IV,date
0,WBA,0.874193,2024-10-15
1,DFS,0.744820,2024-10-16
2,UAL,0.714157,2024-10-15
3,POOL,0.709247,2024-10-17
4,TFC,0.658761,2024-10-17


In [46]:
def getSectorBetas():
    sectorList = ['XLK','XLV','XLF','XLY','XLP','XLE','XLI','XLB','XLU','XLRE','XLC']

    sectorsBeta = {}
    for sector in sectorList:
        sectorProfile = volt.getCompanyProfile(sector)['profile']
        sectorBeta = sectorProfile['beta']
        sectorName = sectorProfile['companyName'].replace(' Select Sector SPDR\xa0Fund','').replace(' Select Sector SPDR Fund','')

        sectorsBeta[sectorName] = sectorBeta

    return str(sectorsBeta)

In [47]:
getSectorBetas()

"{'Technology': 1.22, 'Health Care': 0.67, 'Financial': 1.02, 'Consumer Discretionary': 1.29, 'Consumer Staples': 0.58, 'Energy': 0.72, 'Industrial': 1.09, 'Materials': 1.17, 'Utilities': 0.74, 'The Real Estate': 1.19, 'Communication Services': 1.1}"

In [4]:
import datetime
def getMarketSnapshot():
    vix = volt.getMarketIndex('VIX')['price']
    vixWeekAgo = volt.getPrices('^VIX').iloc[5]['close']
    vixChange = (vix - vixWeekAgo) / vixWeekAgo

    Vvix = volt.getMarketIndex('VVIX')['price']
    VvixWeekAgo = volt.getPrices('^VVIX').iloc[5]['close']
    VvixChange = (Vvix - VvixWeekAgo) / VvixWeekAgo

    spHistVolatility = volt.getHistoricalVolatility('^SPX',30)
    weekAgo = datetime.datetime.today() - datetime.timedelta(7)
    spHistoricalVolatilityWeekAgo = volt.getHistoricalVolatility('^SPX',30,weekAgo)
    spChange = (spHistVolatility - spHistoricalVolatilityWeekAgo) / spHistoricalVolatilityWeekAgo
    
    returnDict = {
        'VIX': {
            'currentVIX': vix,
            'Weekly Change': str(vixChange * 100) + '%'
        },
        'S&P500': {
            'Realized Volatility': str(spHistVolatility * 100) + '%',
            'Realized Volatility Weekly Change': str(spChange * 100) + '%'
        },
        'VVIX': {
            'currentVVIX': Vvix,
            'Weekly Change': str(VvixChange * 100) + '%'
        }
    }

    return str(returnDict)

getMarketSnapshot()


"{'VIX': {'currentVIX': 21.1, 'Weekly Change': '9.838625715773038%'}, 'S&P500': {'Realized Volatility': '9.852167083706673%', 'Realized Volatility Weekly Change': '-14.506347276239767%'}, 'VVIX': {'currentVVIX': 117.43, 'Weekly Change': '1.5479072985126308%'}}"

In [8]:
def getMacroRisk():
    vix = volt.getMarketIndex('VIX')
    vixWeekAgo = volt.getPrices('^VIX').iloc[5]['close']
    vixChange = (vix['price'] - vixWeekAgo)

    returnDict = {
        'Current VIX': vix['price'],
        'Weekly Change': vixChange,
        'Weekly Change Percentage': str((vixChange / vixWeekAgo) * 100) + '%',
        '52-week High': vix['yearHigh'],
        '52-week Low': vix['yearLow']
    }
    return str(returnDict)

"{'Current VIX': 20.87, 'Weekly Change': 1.6600000000000001, 'Weekly Change Percentage': '8.64133263925039%', '52-week High': 35.05, '52-week Low': 15.53}"

In [18]:
def getHighImpactBetas():
    highImpactStocksSample = pd.concat([
        volt.getCompaniesETF('XLF')['asset'][0:15],
        volt.getCompaniesETF('XLK')['asset'][0:5],
        volt.getCompaniesETF('XLV')['asset'][0:3]
    ])

    returnDict = {}
    highImpactStocks = highImpactStocksSample.sample(5)
    for stock in highImpactStocks:
        profile = volt.getCompanyProfile(stock)['profile']
        returnDict[f"{profile['companyName']} ({profile['symbol']})"] = profile['beta']

    return str(returnDict)

In [19]:
getHighImpactBetas()

"{'Blackstone Inc. (BX)': 1.502, 'Broadcom Inc. (AVGO)': 1.184, 'Visa Inc. (V)': 0.961, 'Morgan Stanley (MS)': 1.356, 'BlackRock, Inc. (BLK)': 1.318}"

In [67]:
def getInterestRateEnviroment():
    tresuryRates = volt.getTresuryRates()
    today = tresuryRates.iloc[0].to_dict()
    week = tresuryRates.iloc[5].to_dict()

    month1 = tresuryRates.iloc[abs(pd.to_datetime(tresuryRates['date']) - pd.Timestamp(datetime.datetime.today() - datetime.timedelta(30))).idxmin()].to_dict()

    month3 = tresuryRates.iloc[-1].to_dict()

    return {
        'Treasury Rates': {
            'Today': today,
            'One Week Ago': week,
            'One Month Ago': month1,
            'Three Months Ago': month3
        }
    }

In [15]:
import datetime

def getHighImpactEconomicEvents():
    economicCalendar = volt.getEconomicsCalendar()

    highImpactEvents = economicCalendar[economicCalendar['impact'] == 'High'].reset_index(drop=True)
    highImpactEvents['date'] = pd.to_datetime(highImpactEvents['date'])

    future_date = pd.Timestamp(datetime.datetime.today() + datetime.timedelta(days=7))
    index = abs(highImpactEvents['date'] - future_date).idxmin() + 1 

    highImpactEventsThisWeek = highImpactEvents[0:index]
    highImpactEventsThisWeek.drop(columns=['change','actual','impact','changePercentage','unit'],inplace=True)

    returnDict = {}
    for index, row in highImpactEventsThisWeek.iterrows():
        rowDict = row.to_dict()
        rowDict['date'] = str(rowDict['date'])
        event = rowDict.pop('event')
        returnDict[event] = rowDict

    return str(returnDict)


In [16]:
getHighImpactEconomicEvents()

/var/folders/vk/jptmnx617sn4sx4kl14zfspr0000gn/T/ipykernel_47347/2561485424.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  highImpactEventsThisWeek.drop(columns=['change','actual','impact','changePercentage','unit'],inplace=True)


"{'NY Empire State Manufacturing Index (Oct)': {'date': '2024-10-15 12:30:00', 'previous': nan, 'estimate': nan}, 'Continuing Jobless Claims (Oct/05)': {'date': '2024-10-17 12:30:00', 'previous': 1861.0, 'estimate': 1870.0}, 'Jobless Claims 4-week Average (Oct/12)': {'date': '2024-10-17 12:30:00', 'previous': 231.0, 'estimate': 232.0}, 'Initial Jobless Claims (Oct/12)': {'date': '2024-10-17 12:30:00', 'previous': 258.0, 'estimate': 241.0}, 'Retail Sales Ex Gas/Autos MoM (Sep)': {'date': '2024-10-17 12:30:00', 'previous': 0.2, 'estimate': 0.1}, 'Retail Sales Ex Autos MoM (Sep)': {'date': '2024-10-17 12:30:00', 'previous': 0.1, 'estimate': 0.1}, 'Retail Sales YoY (Sep)': {'date': '2024-10-17 12:30:00', 'previous': 2.1, 'estimate': 1.6}, 'Retail Sales MoM (Sep)': {'date': '2024-10-17 12:30:00', 'previous': 0.1, 'estimate': 0.3}, 'Building Permits (Sep)': {'date': '2024-10-18 12:30:00', 'previous': 1.47, 'estimate': 1.46}}"

In [14]:
def highImpactStocksIV():
    spy = volt.getCompaniesETF('SPY')['asset'].tolist()

    spyImpliedVolatilities = volt.get30dayIVList(spy)
    df = pd.DataFrame(list(spyImpliedVolatilities.items()),columns=['ticker','IV'])
    df = df.dropna()
    df = df[df['IV'] != 1.0].sort_values('IV',ascending=False)

    return df

['AAPL', 'NVDA', 'MSFT', 'AMZN', 'META', 'GOOGL', 'AVGO', 'BRK-B', 'GOOG', 'LLY', 'TSLA', 'JPM', 'UNH', 'XOM', 'V', 'MA', 'HD', 'PG', 'COST', 'JNJ', 'WMT', 'ABBV', 'NFLX', 'CRM', 'ORCL', 'MRK', 'BAC', 'KO', 'AMD', 'CVX', 'PEP', 'TMO', 'ACN', 'LIN', 'ADBE', 'MCD', 'CSCO', 'IBM', 'GE', 'ABT', 'WFC', 'NOW', 'CAT', 'QCOM', 'PM', 'TXN', 'VZ', 'INTU', 'AMGN', 'DHR', 'ISRG', 'AMAT', 'DIS', 'NEE', 'PFE', 'SPGI', 'RTX', 'UBER', 'GS', 'CMCSA', 'LOW', 'T', 'AXP', 'PGR', 'UNP', 'BKNG', 'HON', 'MS', 'ETN', 'BLK', 'COP', 'TJX', 'BSX', 'LMT', 'VRTX', 'C', 'SYK', 'PANW', 'ADP', 'MU', 'ADI', 'ELV', 'MDT', 'MMC', 'PLD', 'FI', 'BX', 'KLAC', 'CB', 'LRCX', 'SBUX', 'BMY', 'REGN', 'ANET', 'GILD', 'DE', 'TMUS', 'AMT', 'SCHW', 'INTC', 'NKE', 'CI', 'UPS', 'SO', 'MDLZ', 'ICE', 'KKR', 'TT', 'DUK', 'PLTR', 'SHW', 'BA', 'ZTS', 'MO', 'CVS', 'EQIX', 'CEG', 'SNPS', 'CL', 'PH', 'PYPL', 'CMG', 'CME', 'APH', 'MSI', 'WM', 'AON', 'CDNS', 'TDG', 'GD', 'EOG', 'MCO', 'MMM', 'CRWD', 'GEV', 'PNC', 'NOC', 'TGT', 'HCA', 'FCX', 'W

In [74]:
def getImpliedMove(symbol):
    optionsChain = volt.getOptionsChain(symbol=symbol)

    optionDate = volt.getClosestDate(optionsChain,5)
    optionsWithinDate = optionsChain[optionsChain['expiration_date']==optionDate]

    strikePrice = volt.getClosestStrikePrice(optionsWithinDate,symbol=symbol)
    straddle = optionsWithinDate[optionsWithinDate['strike_price'] == strikePrice]

    prices = []
    if len(straddle) != 0:
        for index, option in straddle.iterrows():
            ticker = option['ticker'].replace('O:','')
            price = volt.getMarketValue(ticker)
            if price != np.nan:
                prices.append(price)
        straddlePrice = np.sum(prices)
    else:
        straddlePrice = np.nan

    return straddlePrice / volt.getPrice(symbol)

type(getImpliedMove('NFLX'))



numpy.float64

In [13]:
def getMarketExpectations():
    spyIV = volt.getkdayVolatility('SPY',30) / 100
    qqqIV = volt.getkdayVolatility('QQQ',30) /100

    spyHV = volt.getHistoricalVolatility('SPY',30)
    qqqHV = volt.getHistoricalVolatility('QQQ',30)

    if spyIV == 1.0 or qqqIV == 1.0 or abs(spyIV) == np.inf or abs(qqqIV) == np.inf or spyIV == np.nan or qqqIV == np.nan:
        spyIV = 'unable to retrieve'
        qqqIV = 'unable to retrieve'
        spyIVHVratio =  spyHV
        QQQIVHVratio =   qqqHV
    else:
        spyIVHVratio = spyIV / spyHV
        QQQIVHVratio = qqqIV / qqqHV

    volatilities = {
        'SPY': {
            'Current Implied Volatility': spyIV,
            'Current 30-day Historical Volatility': spyHV,
            'IV/HV Ratio': spyIVHVratio
        },
        'QQQ': {
            'Current Implied Volatility': qqqIV,
            'Current 30-day Historical Volatility': qqqHV,
            'IV/HV Ratio': QQQIVHVratio
        }
    }

    return str(volatilities)

getMarketExpectations()

"{'SPY': {'Current Implied Volatility': 0.16075000000000003, 'Current 30-day Historical Volatility': 0.09602127212665308, 'IV/HV Ratio': 1.674108209980483}, 'QQQ': {'Current Implied Volatility': 0.213, 'Current 30-day Historical Volatility': 0.1357518930787495, 'IV/HV Ratio': 1.5690388927131866}}"

In [8]:
def generateMarketVolatilityIndexes() -> str:
    ticker = 'SPY'

    monthIV = round(volt.getkdayVolatility(ticker,30),2)
    twoMonthIV = round(volt.getkdayVolatility(ticker,60),2)
    threeMonthIV = round(volt.getkdayVolatility(ticker,90),2)
    
    termStructre = {
        'Volatility Term Structure SPY': {
            '30-day IV': monthIV,
            '60-day IV': twoMonthIV,
            '90-day IV': threeMonthIV
        }
    }

    return str(termStructre)

generateMarketVolatilityIndexes()

"{'Volatility Term Structure SPY': {'30-day IV': 16.08, '60-day IV': 15.74, '90-day IV': 15.47}}"